In [1]:
import os
import rasterio as rio
import numpy as np
import shapely
import pyproj
import geopandas as gpd
import matplotlib.pyplot as plt
import rioxarray as riox
import rasterio as rio
import xarray as xr
import netCDF4
from osgeo import gdal
import pandas as pd
from datetime import datetime
import snowFun
import dask.array
import importlib
# %matplotlib widget

# define folder and file paths
folder_AGVA = os.path.join('C:',os.sep,'Users','lzell','OneDrive - Colostate','Desktop',"AGVA")
folder_dems = os.path.join(folder_AGVA, "DEMs", "time_varying_DEMs", "10m")
folder_class = os.path.join(folder_AGVA, 'classified images', 'S2_Classified_Cloudmasked_Merged')
folder_cloud = os.path.join(folder_AGVA, 'classified images', 'S2_Cloud_Merged')
folder_meta = os.path.join(folder_AGVA, "classified images", "meta csv", "S2")
folder_mask = os.path.join(folder_AGVA, 'Derived products', 'S2', 'Masks')

# open rgi
path_rgi = os.path.join(folder_AGVA, 'RGI', "01_rgi60_Alaska", "01_rgi60_Alaska.shp")
rgi_gdf = gpd.read_file(path_rgi)

In [2]:
### iterate through glaciers, running the analysis for each
# get rgi names for given o2 region
rgis_o2 = rgi_gdf[rgi_gdf['O2Region']=='4']['RGIId'].values

# load rgi names that have been saved to the classified folder
rgis_folder = list(set( [ i[3:17] for i in os.listdir(folder_class) if i!='merged.vrt' ] ))

# select which rgis to analyze
# rgis_to_analyze = ["RGI60-01.09162"] # just a single rgi
# rgis_to_analyze = rgis_folder # everything that is available
rgis_to_analyze = list( set(rgis_folder).intersection(set(rgis_o2)) ) # all the rgis in the folder than are in this o2region

# sort
rgis_to_analyze.sort()
# print(rgis_to_analyze)

In [4]:
skip = 0
importlib.reload(snowFun)
for i in range(len(rgis_to_analyze)):
    
    # subset rgi to single outline, by choosing rgiid or rgi name
    rgiid = rgis_to_analyze[i]
    
#     if (i+1)!=243: continue
#     if rgiid != "RGI60-01.11350": continue
#     if skip: continue

    # grab just this rgi geometry and info
    rgi_single = rgi_gdf[rgi_gdf['RGIId']==rgiid].to_crs("EPSG:3338")
    single_geometry = rgi_single.geometry
    # single_geometry = single_geometry.buffer(-100) #what if we buffer out the exterior 100 meters of the glacier
    
    # quickly grab glacier area
    ga = rgi_gdf[rgi_gdf['RGIId']==rgiid]['Area']
    
    print(f"\nStarting {i+1} of {len(rgis_to_analyze)}: {rgiid}  {ga.values[0]} km2")

    # open glacier mask
    glacier_mask = xr.open_dataset(os.path.join(folder_mask, f"S2_{rgiid}_mask.nc"), chunks='auto')   
        
    ### lastly calculate elas for each year, pick the best, save all our products
    csv_data = []
    # xr_data = []
#     chunks = {'time':-1, 'y':50, 'x':50}
    chunks = {'time':1, 'y':-1, 'x':-1}
    # chunks = {'time':-1, 'y':10, 'x':10}

    # we'll store everything in a single csv (all 5 years)
    snow_dfs = []
    obs_dfs = []
    
    for y in [2018,2019,2020,2021,2022]: #2018,2019,2020,2021,2022
#         print(y)

        # open the data. think about how you want to chunk it
        folder_save = os.path.join(folder_AGVA, 'Derived products', 'S2')
        path_open = os.path.join(folder_save, 'Daily AAs', f"S2_{rgiid}_{y}_daily_AAs_smoothed.nc")
        snow = xr.open_dataset(path_open, chunks=chunks)['class'] # nodata=0, ablation=1, snow=2
    
        # make snow to nan, 0, 1 (instead of 0,1,2) 
        snow = xr.where(snow==0, np.nan, snow-1)

        # open dem
        xr_dem = snowFun.get_year_DEM(single_geometry, 2013)

        # shave off edges to make sure dem, mask, snow all match
        xr_dem = xr_dem.sel(x=slice( min(glacier_mask.x.values), max(glacier_mask.x.values) ), y=slice(max(glacier_mask.y.values),min(glacier_mask.y.values)))
        glacier_mask = glacier_mask.sel(x=slice( min(xr_dem.x.values), max(xr_dem.x.values) ), y=slice(max(xr_dem.y.values),min(xr_dem.y.values)))
        snow = snow.sel(x=slice( min(xr_dem.x.values), max(xr_dem.x.values) ), y=slice(max(xr_dem.y.values),min(xr_dem.y.values)))

        print(f'starting ELAs {y}')

        
        # extract number of snow pixels in bands from each time step
        scas = snowFun.extract_band_SCA(snow, xr_dem, glacier_mask, step=10)
        snow_dfs.append(scas)
        
        # extract number of pixels observed in bands from each time step
        scas = snowFun.extract_band_SCA( xr.where(snow>=0, 1, 0) , xr_dem, glacier_mask, step=10)
        obs_dfs.append(scas)

    # concat all the years together, save to csv
    meta_data = snow_dfs[0][['z_min','z_max','total_pixels']]
    snow_dfs = [meta_data] + [ df.drop(['z_min','z_max','total_pixels'], axis=1) for df in snow_dfs ]
    result_df = pd.concat(snow_dfs, ignore_index=False, sort=False, axis=1)
    
    # save to csv
    path_save = os.path.join(folder_AGVA, 'Derived products', 'S2', 'Band SCFs', f"S2_{rgiid}_snow.csv")
    result_df.to_csv(path_save, index=True)
    
    
    # concat all the years together, save to csv
    meta_data = obs_dfs[0][['z_min','z_max','total_pixels']]
    obs_dfs = [meta_data] + [ df.drop(['z_min','z_max','total_pixels'], axis=1) for df in obs_dfs ]
    result_df = pd.concat(obs_dfs, ignore_index=False, sort=False, axis=1)
    
    # save to csv
    path_save = os.path.join(folder_AGVA, 'Derived products', 'S2', 'Band SCFs', f"S2_{rgiid}_observed.csv")
    result_df.to_csv(path_save, index=True)
        


Starting 1 of 483: RGI60-01.01546  6.078 km2
starting ELAs 2018
starting ELAs 2019
starting ELAs 2020
starting ELAs 2021
starting ELAs 2022

Starting 2 of 483: RGI60-01.01560  3.87 km2
starting ELAs 2018
starting ELAs 2019
starting ELAs 2020
starting ELAs 2021
starting ELAs 2022

Starting 3 of 483: RGI60-01.01573  2.809 km2
starting ELAs 2018
starting ELAs 2019
starting ELAs 2020
starting ELAs 2021
starting ELAs 2022

Starting 4 of 483: RGI60-01.01589  2.757 km2
starting ELAs 2018
starting ELAs 2019
starting ELAs 2020
starting ELAs 2021
starting ELAs 2022

Starting 5 of 483: RGI60-01.01602  4.242 km2
starting ELAs 2018
starting ELAs 2019
starting ELAs 2020
starting ELAs 2021
starting ELAs 2022

Starting 6 of 483: RGI60-01.01606  2.629 km2
starting ELAs 2018
starting ELAs 2019
starting ELAs 2020
starting ELAs 2021
starting ELAs 2022

Starting 7 of 483: RGI60-01.01607  9.649 km2
starting ELAs 2018
starting ELAs 2019
starting ELAs 2020
starting ELAs 2021
starting ELAs 2022

Starting 8 of

starting ELAs 2021
starting ELAs 2022

Starting 59 of 483: RGI60-01.01819  2.496 km2
starting ELAs 2018
starting ELAs 2019
starting ELAs 2020
starting ELAs 2021
starting ELAs 2022

Starting 60 of 483: RGI60-01.01821  49.911 km2
starting ELAs 2018
starting ELAs 2019
starting ELAs 2020
starting ELAs 2021
starting ELAs 2022

Starting 61 of 483: RGI60-01.01822  3.351 km2
starting ELAs 2018
starting ELAs 2019
starting ELAs 2020
starting ELAs 2021
starting ELAs 2022

Starting 62 of 483: RGI60-01.01828  9.383 km2
starting ELAs 2018
starting ELAs 2019
starting ELAs 2020
starting ELAs 2021
starting ELAs 2022

Starting 63 of 483: RGI60-01.01834  2.92 km2
starting ELAs 2018
starting ELAs 2019
starting ELAs 2020
starting ELAs 2021
starting ELAs 2022

Starting 64 of 483: RGI60-01.01835  8.09 km2
starting ELAs 2018
starting ELAs 2019
starting ELAs 2020
starting ELAs 2021
starting ELAs 2022

Starting 65 of 483: RGI60-01.01836  4.078 km2
starting ELAs 2018
starting ELAs 2019
starting ELAs 2020
startin

starting ELAs 2019
starting ELAs 2020
starting ELAs 2021
starting ELAs 2022

Starting 117 of 483: RGI60-01.09173  14.846 km2
starting ELAs 2018
starting ELAs 2019
starting ELAs 2020
starting ELAs 2021
starting ELAs 2022

Starting 118 of 483: RGI60-01.09177  5.618 km2
starting ELAs 2018
starting ELAs 2019
starting ELAs 2020
starting ELAs 2021
starting ELAs 2022

Starting 119 of 483: RGI60-01.09178  2.754 km2
starting ELAs 2018
starting ELAs 2019
starting ELAs 2020
starting ELAs 2021
starting ELAs 2022

Starting 120 of 483: RGI60-01.09199  3.095 km2
starting ELAs 2018
starting ELAs 2019
starting ELAs 2020
starting ELAs 2021
starting ELAs 2022

Starting 121 of 483: RGI60-01.09201  8.66 km2
starting ELAs 2018
starting ELAs 2019
starting ELAs 2020
starting ELAs 2021
starting ELAs 2022

Starting 122 of 483: RGI60-01.09202  4.089 km2
starting ELAs 2018
starting ELAs 2019
starting ELAs 2020
starting ELAs 2021
starting ELAs 2022

Starting 123 of 483: RGI60-01.09209  2.597 km2
starting ELAs 2018

starting ELAs 2021
starting ELAs 2022

Starting 174 of 483: RGI60-01.09571  4.47 km2
starting ELAs 2018
starting ELAs 2019
starting ELAs 2020
starting ELAs 2021
starting ELAs 2022

Starting 175 of 483: RGI60-01.09581  25.839 km2
starting ELAs 2018
starting ELAs 2019
starting ELAs 2020
starting ELAs 2021
starting ELAs 2022

Starting 176 of 483: RGI60-01.09592  14.225 km2
starting ELAs 2018
starting ELAs 2019
starting ELAs 2020
starting ELAs 2021
starting ELAs 2022

Starting 177 of 483: RGI60-01.09599  3.891 km2
starting ELAs 2018
starting ELAs 2019
starting ELAs 2020
starting ELAs 2021
starting ELAs 2022

Starting 178 of 483: RGI60-01.09606  5.624 km2
starting ELAs 2018
starting ELAs 2019
starting ELAs 2020
starting ELAs 2021
starting ELAs 2022

Starting 179 of 483: RGI60-01.09608  12.677 km2
starting ELAs 2018
starting ELAs 2019
starting ELAs 2020
starting ELAs 2021
starting ELAs 2022

Starting 180 of 483: RGI60-01.09616  3.042 km2
starting ELAs 2018
starting ELAs 2019
starting ELAs 20


Starting 231 of 483: RGI60-01.10075  7.597 km2
starting ELAs 2018
starting ELAs 2019
starting ELAs 2020
starting ELAs 2021
starting ELAs 2022

Starting 232 of 483: RGI60-01.10079  3.117 km2
starting ELAs 2018
starting ELAs 2019
starting ELAs 2020
starting ELAs 2021
starting ELAs 2022

Starting 233 of 483: RGI60-01.10098  2.584 km2
starting ELAs 2018
starting ELAs 2019
starting ELAs 2020
starting ELAs 2021
starting ELAs 2022

Starting 234 of 483: RGI60-01.10108  23.944 km2
starting ELAs 2018
starting ELAs 2019
starting ELAs 2020
starting ELAs 2021
starting ELAs 2022

Starting 235 of 483: RGI60-01.10121  28.184 km2
starting ELAs 2018
starting ELAs 2019
starting ELAs 2020
starting ELAs 2021
starting ELAs 2022

Starting 236 of 483: RGI60-01.10134  6.974 km2
starting ELAs 2018
starting ELAs 2019
starting ELAs 2020
starting ELAs 2021
starting ELAs 2022

Starting 237 of 483: RGI60-01.10157  2.904 km2
starting ELAs 2018
starting ELAs 2019
starting ELAs 2020
starting ELAs 2021
starting ELAs 20

starting ELAs 2018
starting ELAs 2019
starting ELAs 2020
starting ELAs 2021
starting ELAs 2022

Starting 289 of 483: RGI60-01.10683  237.897 km2
starting ELAs 2018
starting ELAs 2019
starting ELAs 2020
starting ELAs 2021
starting ELAs 2022

Starting 290 of 483: RGI60-01.10689  773.873 km2
starting ELAs 2018
starting ELAs 2019
starting ELAs 2020
starting ELAs 2021
starting ELAs 2022

Starting 291 of 483: RGI60-01.10732  11.782 km2
starting ELAs 2018
starting ELAs 2019
starting ELAs 2020
starting ELAs 2021
starting ELAs 2022

Starting 292 of 483: RGI60-01.10756  5.702 km2
starting ELAs 2018
starting ELAs 2019
starting ELAs 2020
starting ELAs 2021
starting ELAs 2022

Starting 293 of 483: RGI60-01.10761  3.26 km2
starting ELAs 2018
starting ELAs 2019
starting ELAs 2020
starting ELAs 2021
starting ELAs 2022

Starting 294 of 483: RGI60-01.10764  8.185 km2
starting ELAs 2018
starting ELAs 2019
starting ELAs 2020
starting ELAs 2021
starting ELAs 2022

Starting 295 of 483: RGI60-01.10768  2.795

starting ELAs 2020
starting ELAs 2021
starting ELAs 2022

Starting 346 of 483: RGI60-01.17659  2.503 km2
starting ELAs 2018
starting ELAs 2019
starting ELAs 2020
starting ELAs 2021
starting ELAs 2022

Starting 347 of 483: RGI60-01.17681  7.526 km2
starting ELAs 2018
starting ELAs 2019
starting ELAs 2020
starting ELAs 2021
starting ELAs 2022

Starting 348 of 483: RGI60-01.17711  2.401 km2
starting ELAs 2018
starting ELAs 2019
starting ELAs 2020
starting ELAs 2021
starting ELAs 2022

Starting 349 of 483: RGI60-01.17720  2.409 km2
starting ELAs 2018
starting ELAs 2019
starting ELAs 2020
starting ELAs 2021
starting ELAs 2022

Starting 350 of 483: RGI60-01.17726  4.334 km2
starting ELAs 2018
starting ELAs 2019
starting ELAs 2020
starting ELAs 2021
starting ELAs 2022

Starting 351 of 483: RGI60-01.17748  7.195 km2
starting ELAs 2018
starting ELAs 2019
starting ELAs 2020
starting ELAs 2021
starting ELAs 2022

Starting 352 of 483: RGI60-01.17761  307.536 km2
starting ELAs 2018
starting ELAs 20

starting ELAs 2021
starting ELAs 2022

Starting 403 of 483: RGI60-01.17995  13.064 km2
starting ELAs 2018
starting ELAs 2019
starting ELAs 2020
starting ELAs 2021
starting ELAs 2022

Starting 404 of 483: RGI60-01.17996  3.476 km2
starting ELAs 2018
starting ELAs 2019
starting ELAs 2020
starting ELAs 2021
starting ELAs 2022

Starting 405 of 483: RGI60-01.17999  35.181 km2
starting ELAs 2018
starting ELAs 2019
starting ELAs 2020
starting ELAs 2021
starting ELAs 2022

Starting 406 of 483: RGI60-01.18004  9.172 km2
starting ELAs 2018
starting ELAs 2019
starting ELAs 2020
starting ELAs 2021
starting ELAs 2022

Starting 407 of 483: RGI60-01.18008  2.07 km2
starting ELAs 2018
starting ELAs 2019
starting ELAs 2020
starting ELAs 2021
starting ELAs 2022

Starting 408 of 483: RGI60-01.18011  2.073 km2
starting ELAs 2018
starting ELAs 2019
starting ELAs 2020
starting ELAs 2021
starting ELAs 2022

Starting 409 of 483: RGI60-01.18019  43.424 km2
starting ELAs 2018
starting ELAs 2019
starting ELAs 20


Starting 460 of 483: RGI60-01.23597  5.327 km2
starting ELAs 2018
starting ELAs 2019
starting ELAs 2020
starting ELAs 2021
starting ELAs 2022

Starting 461 of 483: RGI60-01.23606  2.344 km2
starting ELAs 2018
starting ELAs 2019
starting ELAs 2020
starting ELAs 2021
starting ELAs 2022

Starting 462 of 483: RGI60-01.23635  129.755 km2
starting ELAs 2018
starting ELAs 2019
starting ELAs 2020
starting ELAs 2021
starting ELAs 2022

Starting 463 of 483: RGI60-01.23636  3.9 km2
starting ELAs 2018
starting ELAs 2019
starting ELAs 2020
starting ELAs 2021
starting ELAs 2022

Starting 464 of 483: RGI60-01.23675  2.499 km2
starting ELAs 2018
starting ELAs 2019
starting ELAs 2020
starting ELAs 2021
starting ELAs 2022

Starting 465 of 483: RGI60-01.23679  7.983 km2
starting ELAs 2018
starting ELAs 2019
starting ELAs 2020
starting ELAs 2021
starting ELAs 2022

Starting 466 of 483: RGI60-01.23682  2.139 km2
starting ELAs 2018
starting ELAs 2019
starting ELAs 2020
starting ELAs 2021
starting ELAs 2022

In [ ]:
tots = scas['total_pixels']
temp = scas.drop(['band_n','z_min','z_max','total_pixels'], axis=1)

image = temp.to_numpy() / np.expand_dims(tots.to_numpy(), axis=1) 
plt.imshow(image)
plt.colorbar()